In [3]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [4]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [6]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = llm | output_parser
result = chain.invoke("Tell me a joke")
print(result)

Why did the scarecrow win an award? 

Because he was outstanding in his field!


In [ ]:
'''from typing import List
from pydantic import BaseModel, Field

class MobileReview(BaseModel):
    phone_model: str = Field(description="Name and model of the phone")
    rating: float = Field(description="Overall rating out of 5")
    pros: List[str] = Field(description="List of positive aspects")
    cons: List[str] = Field(description="List of negative aspects")
    summary: str = Field(description="Brief summary of the review")

review_text = """
Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous,
colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
stronger. Battery life's solid, lasts me all day no problem.
Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
Also, still getting used to the new button layout, keep hitting Bixby by mistake.
Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
being perfect. If you're due for an upgrade, definitely worth checking out!
"""

structured_llm = llm.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)
print(output)
print(output.pros)'''

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
chain = prompt | llm | output_parser
result = chain.invoke({"topic": "programming"})
print(result)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

Human_Message = ""
messages = [
    SystemMessage(content="You are a helpful assistant that tells jokes."),
    HumanMessage(content="{Human_Message}")
]
response = llm.invoke(messages)
print(response)

template = ChatPromptTemplate([
    ("system", "You are a helpful assistant that tells jokes."),
    ("human", "Tell me about {topic}")
])
chain = template | llm
response = chain.invoke({"topic": "programming"})
print(response)

In [12]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
import os

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

folder_path = "/Users/viditparashar/Documents/Papers"
documents = load_documents(folder_path)
print(f"Loaded {len(documents)} documents from the folder.")


Loaded 19 documents from the folder.


In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")

Split the documents into 112 chunks.


In [14]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])
print(document_embeddings[0][:5])


/var/folders/j5/n1bxjzvx5g5bjvl_cp_bxhgh0000gn/T/ipykernel_12596/851758213.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[-0.059489138424396515, -0.02193422242999077, -0.04421456903219223, -0.008920797146856785, 0.025547431781888008]


In [15]:
from langchain_chroma import Chroma

collection_name = "my_collection"
vectorstore = Chroma.from_documents(
    collection_name=collection_name,
    documents=splits,
    embedding=embedding_function,
    persist_directory="./chroma_db"
)
print("Vector store created and persisted to './chroma_db'")


Vector store created and persisted to './chroma_db'


In [16]:
query = ""
search_results = vectorstore.similarity_search(query, k=2)
print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()


Top 2 most relevant chunks for the query: 'LSTM'

Result 1:
Source: /Users/viditparashar/Documents/Papers/1-s2.0-S1877050922020993-main-2.pdf
Content: the previous step, thereby solving the issues of long -term dependencies of RNN where  the RNN give precise 
predictions on rece nt information but are incapable of predicting data stored in long term memory. Some  of the 
major applications of LSTM are captioning images, generation of handwriting chatbots answering questions, and 
various others. [45]

Result 2:
Source: /Users/viditparashar/Documents/Papers/1-s2.0-S1877050922020993-main-2.pdf
Content: the previous step, thereby solving the issues of long -term dependencies of RNN where  the RNN give precise 
predictions on rece nt information but are incapable of predicting data stored in long term memory. Some  of the 
major applications of LSTM are captioning images, generation of handwriting chatbots answering questions, and 
various others. [45] 
 
 Author name / Procedia Computer 

In [17]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever_results = retriever.invoke(query)
print(retriever_results)

[Document(id='ec27c7f7-78c0-4978-95e9-ab1a98ee001e', metadata={'page': 7, 'page_label': '254', 'source': '/Users/viditparashar/Documents/Papers/1-s2.0-S1877050922020993-main-2.pdf'}, page_content='the previous step, thereby solving the issues of long -term dependencies of RNN where  the RNN give precise \npredictions on rece nt information but are incapable of predicting data stored in long term memory. Some  of the \nmajor applications of LSTM are captioning images, generation of handwriting chatbots answering questions, and \nvarious others. [45]'), Document(id='438441e4-eeb3-40e2-ad3a-0de307e7f283', metadata={'page': 8, 'page_label': '255', 'source': '/Users/viditparashar/Documents/Papers/1-s2.0-S1877050922020993-main-2.pdf'}, page_content='the previous step, thereby solving the issues of long -term dependencies of RNN where  the RNN give precise \npredictions on rece nt information but are incapable of predicting data stored in long term memory. Some  of the \nmajor applications of

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """Answer the question based only on the following context:
{context}
Question: {question}
Answer: """

prompt = ChatPromptTemplate.from_template(template)

def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [19]:
question = ""
response = rag_chain.invoke(question)
print(f"Question: {question}")
print(f"Answer: {response}")

Question: How is LSTM used in stock prediction?
Answer: The context does not provide specific details on how Long Short-Term Memory (LSTM) is used in stock prediction. It only mentions that the DL (Deep Learning) algorithm, which includes LSTM, outperforms other algorithms for stock price or time series prediction with extensive accuracy. For detailed methods or applications of LSTM in stock prediction, further information would be required.


In [28]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain

contextualize_q_system_prompt = """
Given a chat history and the latest user question
which might reference context in the chat history,
formulate a standalone question which can be understood
without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

contextualize_chain = contextualize_q_prompt | llm | StrOutputParser()
print(contextualize_chain.invoke({"input": "How to predict stock??", "chat_history": []}))


What methods can be used to predict stock market trends?


In [31]:
from langchain.chains import create_retrieval_chain

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [32]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
question1 = "Is LSTM used in it?"
answer1 = rag_chain.invoke({"input": question1, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question1),
    AIMessage(content=answer1)
])

print(f"Human: {question1}")
print(f"AI: {answer1}\n")

question2 = "if not lstm then what is used?"
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question2),
    AIMessage(content=answer2)
])

print(f"Human: {question2}")
print(f"AI: {answer2}")

Human: Is LSTM used in it?
AI: Yes, Long Short-Term Memory (LSTM) is used in this context. Some of its major applications mentioned include captioning images, generating handwriting, and enabling chatbots to answer questions.

Human: if not lstm then what is used?
AI: The context provided only mentions the use of the Long Short-Term Memory (LSTM) algorithm. If not LSTM, other types of recurrent neural networks (RNNs) or different machine learning algorithms could potentially be used, but the specific alternative is not mentioned in the provided context.


In [38]:
#Multi-User

import sqlite3
from datetime import datetime
import uuid

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute(''' CREATE TABLE IF NOT EXISTS application_logs( id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id TEXT,
    user_query TEXT,
    gpt_response TEXT,
    model TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()
    
def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs(session_id, user_query, gpt_response, model) VALUES(?, ?, ?, ?)', (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()
    
def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human","content": row["user_query"]},
            {"role": "human","content": row["gpt_response"]}
        ])
    conn.close()
    
    return messages

create_application_logs()

In [41]:
session_id = str(uuid.uuid4())
question = "what is this paper about"
chat_history = get_chat_history(session_id)
answer = rag_chain.invoke({"input": question, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question, answer, "gpt-4o-mini")
print(f"Human: {question}")
print(f"AI: {answer}")

Human: what is this paper about
AI: This paper is about the stock market, specifically discussing the various factors that can cause drastic oscillations in stock prices. These factors can include politics, brand reputation, and the current global scenario. The paper emphasizes that while understanding these factors is essential, they are not sufficient for developing a method for accurate prediction of stock market trends due to the ever-changing global conditions.


In [42]:
question2 = "how can i use it?"
chat_history = get_chat_history(session_id)
answer = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question2, answer, "gpt-4o-mini")
print(f"Human: {question2}")
print(f"AI: {answer}")

Human: how can i use it?
AI: This paper can be used to gain a deeper understanding of the factors that influence stock market trends. If you're an investor, you can use this knowledge to make more informed decisions about your investments. If you're a researcher or a student, this paper can serve as a reference for studies on stock market trends and predictions. Furthermore, the paper discusses the use of machine learning techniques for stock market prediction, which could be of interest if you're exploring technological solutions for investment strategies.
